In [1]:
import sys
sys.path.append('..')

import os
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import gaussian, threshold_otsu
from skimage.transform import resize
from skimage import io
import pickle
from skimage.measure import regionprops

import matplotlib.pyplot as plt
import torch
import time
import model.model_cnnvae_conditional
import model.optimizer as optimizer
from sklearn.decomposition import PCA
import pandas as pd
import torchvision

from PIL import Image 

import scanpy
import anndata as ad
import pandas as pd

In [2]:
nProt=6
holdOutSamples_samples=[['HV1','P22','P14','P27','HV3','P46','P24','P37'],
                        ['HV5','P68','P44','P42','HV7','P15','P55','P59'],
                        ['HV8','P47','P63','P70','HV2','P57','P72','P83'],
                        ['HV4','P16','P18','P62','HV6','P52','P41','P38'],
                        ['HV1','P84','P50','P48','HV2','P22','P56','P27'],
                        ['HV3','P46','P14','P37','HV4','P68','P44','P42'],
                        ['HV5','P15','P24','P59','HV6','P47','P55','P70'],
                        ['HV7','P57','P63','P83','HV8','P16','P18','P62'],
                        ['HV1','P52','P72','P38','HV5','P84','P41','P48'],
                        ['HV2','P22','P50','P27','HV3','P68','P24','P59'],
                        ['HV4','P47','P56','P37','HV7','P16','P44','P70'],
                        ['HV6','P46','P14','P42','HV8','P15','P55','P83']]

pnames=np.array(['gh2ax','cd8','cd16', 'cd3', 'cd4',   'lamin'])
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"




In [3]:
sharedSizes=[1024]
dSpecific_filter=[(200,16)]
pID_type='randInit'
pIDemb_size=64

seed=3

name_lord='splitChannels_conditional_lord_withNoise_bce'
modelname_lord='cnn_vae_pbmc_lord'
logsavepath_lord=os.path.join('../../data/c2p/log/',modelname_lord,name_lord)
modelsavepath_lord=os.path.join('../../data/c2p/models/',modelname_lord,name_lord)
plotsavepath_lord=os.path.join('../../data/c2p/plots/',modelname_lord,name_lord)

logsavepath_p_dna=os.path.join(logsavepath_lord,'dna')
modelsavepath_p_dna=os.path.join(modelsavepath_lord,'dna')
plotsavepath_p_dna=os.path.join(plotsavepath_lord,'dna')

logsavepath_p_protein=os.path.join(logsavepath_lord,'protein')
modelsavepath_p_protein=os.path.join(modelsavepath_lord,'protein')
plotsavepath_p_protein=os.path.join(plotsavepath_lord,'protein')

currLatentSize=sharedSizes[0]
dSpecificSize,dfilterSize=dSpecific_filter[0]



In [11]:
def train(epoch,model_clf,optimizer_clf,loss_clf,inputImgs,inputImgs_val,labels_train,labels_val):
    np.random.seed(3)
    train_nodes_idx=np.arange(labels_train.size()[0])
    val_nodes_idx=np.arange(labels_val.size()[0])
    np.random.shuffle(train_nodes_idx)
    t = time.time()
    model_clf.train()
    loss_all=0
    
    ntrainBatches=int(np.ceil(train_nodes_idx.shape[0]/batchsize))
    for i in range(ntrainBatches):
        trainIdx=train_nodes_idx[i*batchsize:min((i+1)*batchsize,train_nodes_idx.shape[0])]
        train_labels=labels_train[trainIdx].cuda().float()
        trainInput=torch.tensor(inputImgs[trainIdx]).cuda().float()

        optimizer_clf.zero_grad()

        pred = model_clf(trainInput)

        loss=loss_clf(pred, train_labels)
        loss_all+=loss.item()
            
        loss.backward()
        optimizer_clf.step()

    loss_all=loss_all/ntrainBatches
    
    with torch.no_grad():
        model_clf.eval()
        loss_val_all=0
        nvalBatches=int(np.ceil(val_nodes_idx.shape[0]/batchsize))
        for i in range(nvalBatches):
            valIdx=val_nodes_idx[i*batchsize:min((i+1)*batchsize,val_nodes_idx.shape[0])]
            val_labels=labels_val[valIdx].cuda().float()
            valInput=torch.tensor(inputImgs_val[valIdx]).cuda().float()


            pred = model_clf(valInput)

            loss=loss_clf(pred, val_labels)
            loss_val_all+=loss.item()

        loss_val_all=loss_val_all/nvalBatches
    
    if epoch%50==0:
        print(' Epoch: {:04d}'.format(epoch),
              'loss_train: {:.4f}'.format(loss_all),
              'loss_val: {:.4f}'.format(loss_val_all),
              'time: {:.4f}s'.format(time.time() - t))
    return loss_all,loss_val_all

In [5]:
epochs=5001
saveFreq=50
lr=0.0001
weight_decay=0

batchsize=256

plotsavepath_c2pf=os.path.join(plotsavepath_lord,'chromatinImg2proteinFeatures')
if not os.path.exists(plotsavepath_c2pf):
    os.mkdir(plotsavepath_c2pf)
logsavepath_c2pf=os.path.join(logsavepath_lord,'chromatinImg2proteinFeatures')
if not os.path.exists(logsavepath_c2pf):
    os.mkdir(logsavepath_c2pf)
modelsavepath_c2pf=os.path.join(modelsavepath_lord,'chromatinImg2proteinFeatures')
if not os.path.exists(modelsavepath_c2pf):
    os.mkdir(modelsavepath_c2pf)
    
name_train='nmcoClf_origImg'
modelname_train='resnet18'

In [24]:
sourceDir='../../data/chromark'
segDir=os.path.join(sourceDir,'nuclear_masks')
imgDir=os.path.join(sourceDir,'raw_data')
conditions=['controls','headneck','meningioma', 'glioma']

outSize=128
savename='pathCentered_'+str(outSize)
p='gh2ax'

for h in range(len(holdOutSamples_samples)):
    print(h)
    holdOutSamples=holdOutSamples_samples[h]

    imgsC_all_val={}
    imgsP_all_val={}
    imgNames_all_val={}
    nmco_all_val={}
    proteinTrainInput_val={}
    pID_all_val={}
    conditions_all_val={}
    for condition_i in conditions:
        segDir_i=os.path.join(segDir,condition_i)
        imgDir_i=os.path.join(imgDir,condition_i)
        for stain in os.listdir(segDir_i):
            segDir_i_stain=os.path.join(segDir_i,stain)
            imgDir_i_stain=os.path.join(imgDir_i,stain)

            segPID2name={}
            for pID_dir in os.listdir(segDir_i_stain):
                pID=pID_dir.split('_')
                segPID2name[pID[0]]=pID_dir
            imgPID2name={}
            for pID_dir in os.listdir(imgDir_i_stain):
                pID=pID_dir.split('_')
                imgPID2name[pID[0]]=pID_dir
            for pID in segPID2name.keys():
                if condition_i=='meningioma' and stain=='dapi_gh2ax_lamin_cd3' and pID=='P33': #skipping incorrect images
                    continue
                if pID in holdOutSamples:
                    print('hold out: '+pID)
                    continue
                if pID not in imgPID2name:
                    continue
                imgDir_i_stain_p=os.path.join(imgDir_i_stain,imgPID2name[pID])
                segDir_i_stain_p=os.path.join(segDir_i_stain,segPID2name[pID])

                with open(os.path.join(imgDir_i_stain_p,savename+'_imgNames'), 'rb') as output:
                    imgNames=pickle.load(output)
                with open(os.path.join(imgDir_i_stain_p,savename+'_img'), 'rb') as output:
                    img=pickle.load(output)
                with open(os.path.join(imgDir_i_stain_p,'nmco_allfeatures_protein'), 'rb') as output:
                    nmco_p=pickle.load(output) #excluding label column

                stain_list=stain.split('_')
                nImgPerStain=int(img.shape[0]/(len(stain_list)-1))
                for sother in range(1,len(stain_list)):
                    
                    if stain_list[sother] not in imgsP_all_val.keys():
                        pID_all_val[stain_list[sother]]=np.repeat(pID,img.shape[0])
                        imgsC_all_val[stain_list[sother]]=img[:,[0]]
                        imgNames_all_val[stain_list[sother]]=imgNames
                        nmco_all_val[stain_list[sother]]=nmco_p[stain_list[sother]]
                        imgsP_all_val[stain_list[sother]]=img[:,[sother]]
                        conditions_all_val[stain_list[sother]]=np.repeat(condition_i,img.shape[0])
                    else:
                        pID_all_val[stain_list[sother]]=np.concatenate((pID_all_val[stain_list[sother]],np.repeat(pID,img.shape[0])))
                        imgsC_all_val[stain_list[sother]]=np.concatenate((imgsC_all_val[stain_list[sother]],img[:,[0]]),axis=0)
                        imgNames_all_val[stain_list[sother]]=np.concatenate((imgNames_all_val[stain_list[sother]],imgNames))
                        nmco_all_val[stain_list[sother]]=np.concatenate((nmco_all_val[stain_list[sother]],nmco_p[stain_list[sother]]),axis=0)
                        imgsP_all_val[stain_list[sother]]=np.concatenate((imgsP_all_val[stain_list[sother]],img[:,[sother]]),axis=0)
                        conditions_all_val[stain_list[sother]]=np.concatenate((conditions_all_val[stain_list[sother]],np.repeat(condition_i,img.shape[0])))
    
    imgsC_val_allProt={}
    imgsP_val_allProt={}
    imgNames_val_allProt={}
    nmco_val_allProt={}
    pID_val_allProt={}
    conditions_val_allProt={}
    for condition_i in conditions:
        segDir_i=os.path.join(segDir,condition_i)
        imgDir_i=os.path.join(imgDir,condition_i)
        for stain in os.listdir(segDir_i):
            segDir_i_stain=os.path.join(segDir_i,stain)
            imgDir_i_stain=os.path.join(imgDir_i,stain)

            segPID2name={}
            for pID_dir in os.listdir(segDir_i_stain):
                pID=pID_dir.split('_')
                segPID2name[pID[0]]=pID_dir
            imgPID2name={}
            for pID_dir in os.listdir(imgDir_i_stain):
                pID=pID_dir.split('_')
                imgPID2name[pID[0]]=pID_dir
            for pID in segPID2name.keys():
                if condition_i=='meningioma' and stain=='dapi_gh2ax_lamin_cd3' and pID=='P33': #skipping incorrect images
                    continue
                if pID not in holdOutSamples:
                    continue
                if pID not in imgPID2name:
                    continue
                imgDir_i_stain_p=os.path.join(imgDir_i_stain,imgPID2name[pID])
                segDir_i_stain_p=os.path.join(segDir_i_stain,segPID2name[pID])

                with open(os.path.join(imgDir_i_stain_p,savename+'_imgNames'), 'rb') as output:
                    imgNames=pickle.load(output)
                with open(os.path.join(imgDir_i_stain_p,savename+'_img'), 'rb') as output:
                    img=pickle.load(output)
                with open(os.path.join(imgDir_i_stain_p,'nmco_allfeatures_protein'), 'rb') as output:
                    nmco=pickle.load(output)

                stain_list=stain.split('_')
                for s in range(1,len(stain_list)):
                    if stain_list[s] not in imgsP_val_allProt.keys():
                        pID_val_allProt[stain_list[s]]=np.repeat(pID,img.shape[0])
                        imgsC_val_allProt[stain_list[s]]=img[:,[0]]
                        imgNames_val_allProt[stain_list[s]]=imgNames
                        nmco_val_allProt[stain_list[s]]=nmco[stain_list[s]]
                        imgsP_val_allProt[stain_list[s]]=img[:,[s]]
                        conditions_val_allProt[stain_list[s]]=np.repeat(condition_i,img.shape[0])
                    else:
                        pID_val_allProt[stain_list[s]]=np.concatenate((pID_val_allProt[stain_list[s]],np.repeat(pID,img.shape[0])))
                        imgsC_val_allProt[stain_list[s]]=np.concatenate((imgsC_val_allProt[stain_list[s]],img[:,[0]]),axis=0)
                        imgNames_val_allProt[stain_list[s]]=np.concatenate((imgNames_val_allProt[stain_list[s]],imgNames))
                        nmco_val_allProt[stain_list[s]]=np.concatenate((nmco_val_allProt[stain_list[s]],nmco[stain_list[s]]),axis=0)
                        imgsP_val_allProt[stain_list[s]]=np.concatenate((imgsP_val_allProt[stain_list[s]],img[:,[s]]),axis=0)
                        conditions_val_allProt[stain_list[s]]=np.concatenate((conditions_val_allProt[stain_list[s]],np.repeat(condition_i,img.shape[0])))

    
    print(p)
    plotsavepath_c2pf_p=os.path.join(plotsavepath_c2pf,p)
    if not os.path.exists(plotsavepath_c2pf_p):
        os.mkdir(plotsavepath_c2pf_p)
    plotsavepath_c2pf_p=os.path.join(plotsavepath_c2pf,p,str(h))
    if not os.path.exists(plotsavepath_c2pf_p):
        os.mkdir(plotsavepath_c2pf_p)
    logsavepath_c2pf_p=os.path.join(logsavepath_c2pf,p)
    if not os.path.exists(logsavepath_c2pf_p):
        os.mkdir(logsavepath_c2pf_p)
    logsavepath_c2pf_p=os.path.join(logsavepath_c2pf,p,str(h))
    if not os.path.exists(logsavepath_c2pf_p):
        os.mkdir(logsavepath_c2pf_p)
    modelsavepath_c2pf_p=os.path.join(modelsavepath_c2pf,p)
    if not os.path.exists(modelsavepath_c2pf_p):
        os.mkdir(modelsavepath_c2pf_p)
    modelsavepath_c2pf_p=os.path.join(modelsavepath_c2pf,p,str(h))
    if not os.path.exists(modelsavepath_c2pf_p):
        os.mkdir(modelsavepath_c2pf_p)
    
        
    nmco_all_finite=np.copy(nmco_all_val[p])
    nmco_all_finite[np.logical_not(np.isfinite(nmco_all_val[p]))]=0
    nmco_all_finite=nmco_all_finite[:,np.sum(np.isfinite(nmco_all_val[p]),axis=0)>0]
    nmco_val_allProt_np_finite=np.copy(nmco_val_allProt[p][:,np.sum(np.isfinite(nmco_all_val[p]),axis=0)>0])
    nmco_val_allProt_np_finite[np.logical_not(np.isfinite(nmco_val_allProt_np_finite))]=0
    
    with open(os.path.join(sourceDir,'nmco_allfeatures_names_'+p), 'rb') as output:
        nmco_names=pickle.load(output)
    nmco_names_finite=nmco_names[np.sum(np.isfinite(nmco_all_val[p]),axis=0)>0]
    
    for nmcoIdx in range(nmco_names_finite.size):
        if nmco_names_finite[nmcoIdx] not in ['kurtosis_gh2ax_2d_int','gh2ax_foci_count','median_gh2ax_3d_int']:
            continue
        print(nmco_names_finite[nmcoIdx])
        
        logsavepath_train_curr=os.path.join(logsavepath_c2pf_p,modelname_train+'_'+name_train+'_'+nmco_names_finite[nmcoIdx])
        modelsavepath_train_curr=os.path.join(modelsavepath_c2pf_p,modelname_train+'_'+name_train+'_'+nmco_names_finite[nmcoIdx])
        plotsavepath_train_curr=os.path.join(plotsavepath_c2pf_p,modelname_train+'_'+name_train+'_'+nmco_names_finite[nmcoIdx])
        if not os.path.exists(logsavepath_train_curr):
            os.mkdir(logsavepath_train_curr)
        if not os.path.exists(modelsavepath_train_curr):
            os.mkdir(modelsavepath_train_curr)
        if not os.path.exists(plotsavepath_train_curr):
            os.mkdir(plotsavepath_train_curr)
            
        np.random.seed(seed)
        pctVal=0.1

        allIdx=np.arange(imgsC_all_val[p].shape[0])
        np.random.shuffle(allIdx)
        valIdx=allIdx[:int(pctVal*allIdx.size)]
        trainIdx=allIdx[int(pctVal*allIdx.size):]

        loss_reg=torch.nn.MSELoss()

        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.enabled = True
        if modelname_train=='resnet18':
            model=torchvision.models.resnet18(num_classes=1)
            model.conv1=torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False) #change to one input channel
        elif modelname_train=='alexnet':
            model=torchvision.models.alexnet(num_classes=1)
            model.features[0]=torch.nn.Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
        elif modelname_train=='cnn_base':
            model=model_clf.cnn_clf.CNN_VAE_clf(4, 2, 1, 1, 64,128,256,256,96, 96*4*4,64,1)
        model.cuda()
        optimizer_clf = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

        train_loss=[np.inf]*epochs
        val_loss=[np.inf]*epochs

        t_ep=time.time()

        if not os.path.exists(os.path.join(logsavepath_train_curr,'val_loss')):
            epCounts=0
            for ep in range(epochs):
                train_loss[ep],val_loss[ep]=train_loss[ep],val_loss[ep]=train(ep,model,optimizer_clf,loss_reg,imgsC_all_val[p][trainIdx],imgsC_all_val[p][valIdx],torch.tensor(nmco_all_finite[trainIdx,nmcoIdx].reshape(-1,1)),torch.tensor(nmco_all_finite[valIdx,nmcoIdx].reshape(-1,1)))

                if ep>50 and val_loss[ep]>=val_loss[ep-50]:
                    epCounts+=1
                else:
                    epCounts=0

                if epCounts>5:
                    break


                if ep%saveFreq == (saveFreq-1):
                    torch.save(model.cpu().state_dict(), os.path.join(modelsavepath_train_curr,'ep'+str(ep)+'.pt'))


                model.cuda()
                torch.cuda.empty_cache()
            print(' total time: {:.4f}s'.format(time.time() - t_ep))


            with open(os.path.join(logsavepath_train_curr,'train_loss'), 'wb') as output:
                pickle.dump(train_loss, output, pickle.HIGHEST_PROTOCOL)
            with open(os.path.join(logsavepath_train_curr,'val_loss'), 'wb') as output:
                pickle.dump(val_loss, output, pickle.HIGHEST_PROTOCOL)
            plt.plot(np.arange(epochs),train_loss)
            plt.plot(np.arange(epochs),val_loss)
            plt.legend(['training loss','validation loss'],loc='upper right')
            plt.savefig(os.path.join(plotsavepath_train_curr,'loss_seed3.jpg'))
            plt.close()
        else:
            with open(os.path.join(logsavepath_train_curr,'val_loss'), 'rb') as output:
                val_loss=pickle.load(output)

        minlossepoch=np.argmin(val_loss)
        minlossepoch_saved=int(np.round(minlossepoch/saveFreq)*saveFreq)-1
        if minlossepoch_saved==-1:
            minlossepoch_saved=saveFreq-1
        if val_loss[minlossepoch_saved-saveFreq]<val_loss[minlossepoch_saved]:
            if val_loss[minlossepoch_saved+saveFreq]<val_loss[minlossepoch_saved-saveFreq]:
                minlossepoch_saved=minlossepoch_saved+saveFreq
            else:
                minlossepoch_saved=minlossepoch_saved-saveFreq
        if minlossepoch_saved==-1:
            minlossepoch_saved=saveFreq-1
        model.load_state_dict(torch.load(os.path.join(modelsavepath_train_curr,'ep'+str(minlossepoch_saved)+'.pt')))

        #validation samples
        with torch.no_grad():
            model.eval()
            loss_val_all=0
            nvalBatches=int(np.ceil(valIdx.shape[0]/batchsize))
            pred_curr=np.array([])
            for i in range(nvalBatches):
                valIdx_i=valIdx[i*batchsize:min((i+1)*batchsize,valIdx.shape[0])]
                val_labels=torch.tensor(nmco_all_finite[i*batchsize:min((i+1)*batchsize,valIdx.shape[0]),nmcoIdx].reshape(-1,1)).cuda().float()
                valInput=torch.tensor(imgsC_all_val[p][valIdx_i]).cuda().float()


                pred = model(valInput)

                loss=loss_reg(pred, val_labels)
                loss_val_all+=loss.item()
                pred_curr=np.concatenate((pred_curr,pred.detach().cpu().numpy().flatten()))

            loss_val_all=loss_val_all/nvalBatches

        print('loss_val: {:.4f}'.format(loss_val_all))
        
        res= pd.DataFrame({'imgNames':imgNames_all_val[p][valIdx],'valIdx':valIdx, 'true':nmco_all_finite[valIdx,nmcoIdx], 'predicted':pred_curr})
        res.to_csv(os.path.join(plotsavepath_train_curr,'predictions_val.csv'))
        
        valIdx=np.arange(imgsC_val_allProt[p].shape[0])
        #heldout samples
        with torch.no_grad():
            model.eval()
            loss_val_all=0
            nvalBatches=int(np.ceil(valIdx.shape[0]/batchsize))
            pred_curr=np.array([])
            for i in range(nvalBatches):
                valIdx_i=valIdx[i*batchsize:min((i+1)*batchsize,valIdx.shape[0])]
                val_labels=torch.tensor(nmco_val_allProt_np_finite[i*batchsize:min((i+1)*batchsize,valIdx.shape[0]),nmcoIdx].reshape(-1,1)).cuda().float()
                valInput=torch.tensor(imgsC_val_allProt[p][valIdx_i]).cuda().float()


                pred = model(valInput)

                loss=loss_reg(pred, val_labels)
                loss_val_all+=loss.item()
                pred_curr=np.concatenate((pred_curr,pred.detach().cpu().numpy().flatten()))

            loss_val_all=loss_val_all/nvalBatches

        print('loss_heldout: {:.4f}'.format(loss_val_all))
        
        res= pd.DataFrame({'imgNames':imgNames_val_allProt[p][valIdx],'valIdx':valIdx, 'true':nmco_val_allProt_np_finite[valIdx,nmcoIdx], 'predicted':pred_curr})
        res.to_csv(os.path.join(plotsavepath_train_curr,'predictions_heldout.csv'))
        
    
    

0
hold out: HV1
hold out: HV3
hold out: HV1
hold out: HV3
hold out: P14
hold out: P24
hold out: P24
hold out: P14
hold out: P37
hold out: P27
hold out: P27
hold out: P37
hold out: P22
hold out: P46
hold out: P46
hold out: P22
gh2ax
kurtosis_gh2ax_2d_int
loss_val: 2507.5735
loss_heldout: 151.4673
gh2ax_foci_count
loss_val: 228.6457
loss_heldout: 113.3196
1
hold out: HV7
hold out: HV5
hold out: HV7
hold out: HV5
hold out: P55
hold out: P44
hold out: P55
hold out: P44
hold out: P59
hold out: P42
hold out: P42
hold out: P59
hold out: P15
hold out: P68
hold out: P68
hold out: P15
gh2ax
kurtosis_gh2ax_2d_int
loss_val: 2356.8674
loss_heldout: 461.3918
gh2ax_foci_count
loss_val: 273.4628
loss_heldout: 72.5066
2
hold out: HV8
hold out: HV2
hold out: HV8
hold out: HV2
hold out: P72
hold out: P63
hold out: P72
hold out: P63
hold out: P83
hold out: P70
hold out: P70
hold out: P83
hold out: P57
hold out: P47
hold out: P57
hold out: P47
gh2ax
kurtosis_gh2ax_2d_int
loss_val: 2436.6695
loss_heldout: 1

In [18]:
with open(os.path.join(logsavepath_train_curr,'train_loss'), 'wb') as output:
    pickle.dump(train_loss, output, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(logsavepath_train_curr,'val_loss'), 'wb') as output:
    pickle.dump(val_loss, output, pickle.HIGHEST_PROTOCOL)
plt.plot(np.arange(epochs),train_loss)
plt.plot(np.arange(epochs),val_loss)
plt.legend(['training loss','validation loss'],loc='upper right')
plt.savefig(os.path.join(plotsavepath_train_curr,'loss_seed3.jpg'))
plt.close()